# Defining your own custom loaders

In [1]:
from rich import print

import cellpy
from cellpy.utils import example_data, plotutils

Defining a simple utility-function to get a peek of the file in question:

In [2]:
def head(f, n=5):
    print(f" {f.name} ".center(80, "="))
    with open(f) as datafile:
        if n > 1:
            for j in range(n):
                line = datafile.readline()
                print(f"[{j+1:02}] {line.rstrip()}")
        else:
            for j, line in enumerate(datafile.readlines()):
                print(f"[{j+1:02}] {line.rstrip()}")
    print(f" {f.name} ".center(80, "="))

## Using the "custom" instrument

This loader can be used if you have simple but unusual files. It needs an instrument file containing a description of the structure of the data file.
You can load files in csv, xlsx, and xls format using this loader.

Here is an example of a custom data file and a corresponding instrument file (yaml format).

In [3]:
p_csv = example_data.custom_file_path()
instrument_file = example_data.custom_instrument_path()

In [4]:
head(p_csv, 30)

=============================== custom_data.csv ================================

[01] # PRIME INSTRUMENT FILE --- M12X---!! HEAD !!---M13B---;;;;;;;;;

[02] number of headers ;19;;;;;;;;

[03] operator;Jan Petter Maehlen;;;;;;;;

[04] date;01.01.2016;;;;;;;;

...

[27] 6;840.01;840.01;43374.42;1;1;0.00;1.14;0.00;0.00

[28] 7;960.01;960.01;43374.43;1;1;0.00;1.14;0.00;0.00

[29] 8;1080.01;1080.01;43374.43;1;1;0.00;1.14;0.00;0.00

[30] 9;1200.01;1200.01;43374.43;1;1;0.00;1.14;0.00;0.00

=============================== custom_data.csv ================================

In [5]:
head(instrument_file, -1)

============================ custom_instrument.yml =============================

[01] ---

[02] formatters:

[03]     skiprows: 19

[04]     sep: ";"

...

[52]     stable_current_hard: 2.0

[53]     stable_current_soft: 4.0

[54]     stable_voltage_hard: 2.0

[55]     stable_voltage_soft: 4.0

============================ custom_instrument.yml =============================

In [6]:
c = cellpy.get(p_csv, instrument="custom", instrument_file=instrument_file)

(cellpy) - self.sep=';', self.skiprows=19, self.header=0, self.encoding='ISO-8859-1', self.decimal='.'
(cellpy) - running post-processor: rename_headers
Index(['index', 'test_time', 'step_time', 'date_stamp', 'step', 'cycle',
       'current', 'voltage', 'charge_capacity', 'discharge_Capacity'],
      dtype='object')
(cellpy) - running post-processor: convert_date_time_to_datetime


In [7]:
plotutils.raw_plot(c, width=1200, height=400)

## Using the "local_instrument" loader

This loader is used for loading data using the corresponding local yaml file with definitions on how the data should be loaded. This loader
is based on the ``TxtLoader`` and can only be used to load csv-type files.
As a "short-cut", this loader will be used if you set the ``instrument`` to the name of the instrument file (with the ``.yml`` extension) e.g.
``c = cellpy.get(rawfile, instrument="instrumentfile.yml")``.
The default instrument file is defined in your cellpy configuration file:
```
Instruments:
  custom_instrument_definitions_file: my_local_instrument.yml
```

As an example, let us see how we could load one of the example Maccor files using a local instrument definition file instead of the implemented "maccor_txt" loader.

In [8]:
p = example_data.maccor_file_path()
print(f"{p.name=}")

p.name='maccor_three.txt'

In [9]:
local_instrument = example_data.local_instrument_path()
print(f"{local_instrument.name=}")

local_instrument.name='local_instrument.yml'

In [10]:
head(local_instrument, -1)

============================= local_instrument.yml =============================

[01] ---

[02] formatters:

[03]     skiprows: 2

[04]     sep: "\t"

...

[66]     stable_current_hard: 2.0

[67]     stable_current_soft: 4.0

[68]     stable_voltage_hard: 2.0

[69]     stable_voltage_soft: 4.0

============================= local_instrument.yml =============================

In [11]:
head(p, 20)

=============================== maccor_three.txt ===============================

[01] Today''s Date      03/28/2022 12:50:27 PM

[02]

[03] Date of Test:      08/23/2021 6:04:18 PM

[04]

...

[17] 12 0       1         0d 00:11:00.00          0d 00:11:00.00        0.0     0.0     0.0     1853.2082       R  
1       08/23/2021 6:15:18 PM

[18] 13 0       1         0d 00:12:00.00          0d 00:12:00.00        0.0     0.0     0.0     1853.0556       R  
1       08/23/2021 6:16:18 PM

[19] 14 0       1         0d 00:13:00.00          0d 00:13:00.00        0.0     0.0     0.0     1853.2082       R  
1       08/23/2021 6:17:18 PM

[20] 15 0       1         0d 00:14:00.00          0d 00:14:00.00        0.0     0.0     0.0     1853.3608       R  
1       08/23/2021 6:18:18 PM

=============================== maccor_three.txt ===============================

In [12]:
from cellpy import log

c = cellpy.get(p, instrument=local_instrument)

(cellpy) - running pre-processor: remove_empty_lines
(cellpy) - self.sep='\t', self.skiprows=2, self.header=0, self.encoding='ISO-8859-1', self.decimal='.'
(cellpy) - running post-processor: rename_headers
Index(['Rec#', 'Cyc#', 'Step', 'TestTime', 'StepTime', 'mAmp-hr', 'mWatt-hr',
       'mAmps', 'Volts', 'State', 'ES', 'DPt Time', 'Unnamed: 12'],
      dtype='object')
(cellpy) - running post-processor: remove_last_if_bad
(cellpy) - running post-processor: split_capacity


(cellpy) - running post-processor: split_current
(cellpy) - running post-processor: set_index
(cellpy) - running post-processor: set_cycle_number_not_zero
(cellpy) - running post-processor: convert_date_time_to_datetime
(cellpy) - running post-processor: convert_step_time_to_timedelta
(cellpy) - running post-processor: convert_test_time_to_timedelta


In [13]:
plotutils.raw_plot(c, width=1200, height=400)

In [14]:
plotutils.summary_plot(c, width=1200, height=400)